### Web Scraping using python and beautiful soup to create a dataframe for futher analysis

#### Import the Libraries needed for this project

In [1]:
#Importing the libraries

import pandas as pd #to create dataframe
import numpy as np  # to count the values 
from bs4 import BeautifulSoup #to get the content in the form of HTML
import requests #The requests module allows you to send HTTP requests using Python
               


#### Create a variable and store the website url that you want to scrape

In [2]:
# Assigning the url to a variable named "url"

url = "https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"

In [3]:
#The requests module allows you to send HTTP requests using Python
#The HTTP request returns a Response Object with all the response data (content, encoding, status, etc)

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
response

<Response [200]>

In [5]:
#creating an empty list, so that we can append the values that we scrape

movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []
Director = []
Stars = []

In [6]:
#Storing the meaningfull required data in the variable "movie_data"

movie_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-advanced' })

In [21]:
#Preview of data that will be scraped

movie_data[0:20]

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt15097216"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt15097216/"> <img alt="Jai Bhim" class="loadlate" data-tconst="tt15097216" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BY2Y5ZWMwZDgtZDQxYy00Mjk0LThhY2YtMmU1MTRmMjVhMjRiXkEyXkFqcGdeQXVyMTI1NDEyNTM5._V1_UY98_CR5,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt15097216/">Jai Bhim</a>
 <span class="lister-item-year text-muted unbold">(2021)</span>
 </h3>
 <p class="text-muted">
 <span class="runtime">164 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Crime, Drama            </span>
 </p>
 <div class="ratings-bar">
 <div class="

In [8]:
# Using a for loop to scrape data using "find" functions and storing the data in the list created as well as doing feature engineering. 

for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
    
    year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(','').replace(')','')
    year.append(year_of_release)
    
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    rate = store.find("div", "inline-block ratings-imdb-rating").text.replace('\n','')
    rating.append(rate)
    
    meta = store.find("span", class_ = "metascore favorable").text.replace(' ', "") if store.find("span", class_ = "metascore favorable") else ''
    metascore.append(meta)
    
    value = store.find_all('span', attrs = {'name' : 'nv'})
    
    vote = value[0].text
    votes.append(vote)
    
    grosses = value[1].text if len(value) > 1 else '******'
    gross.append(grosses)
    
    # Description of the Movies --
    description_1 = store.find_all("p", class_ = "text-muted")
    description_2 = description_1[1].text.replace("\n",'') if len(description_1) > 1 else "******"
    description.append(description_2)
    
    director_1 = store.find_all("p")
    director_2 = director_1[2].text.replace('\n', '').split("|")
    director_3 = director_2[0].split(':')
    director_4 = director_3[1]
    Director.append(director_4)
    
    #Cast Details -- Scraping Director name and Stars 
    cast = director_2[1].split(':')
    cast_1 = cast[1:]
    Stars.append(cast_1)
    
      
   
    
    

In [15]:
#creating a dataframe using pandas library

movie_df = pd.DataFrame({"Name of movie": movie_name, "Year of release": year, "Runtime": time, "Ratings": rating, "Metascore": metascore, "Votes": votes, "Gross": gross, "Discription": description, "Director": Director, "Stars": Stars})

In [11]:
np.count_nonzero(movie_name)

100

In [16]:
# Viewing the first 30 rows of the data

movie_df.head(30)

,Name of movie,Year of release,Runtime,Ratings,Metascore,Votes,Gross,Discription,Director,Stars
0,Jai Bhim,2021,164,9.3,,"170,589",#140,When a tribal man is arrested for a case of al...,T.J. Gnanavel,"[Suriya, Lijo Mol Jose, Manikandan, Rajisha Vi..."
1,The Shawshank Redemption,1994,142,9.3,80,"2,532,350",$28.34M,Two imprisoned men bond over a number of years...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
2,The Godfather,1972,175,9.2,100,"1,743,298",$134.97M,"The Godfather follows Vito Corleone, Don of th...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
3,Soorarai Pottru,2020,153,9.1,,"106,596",******,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Sudha Kongara,"[Suriya, Paresh Rawal, Aparna Balamurali, Prak..."
4,The Dark Knight,2008,152,9.0,84,"2,482,828",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
5,The Godfather: Part II,1974,202,9.0,90,"1,209,449",$57.30M,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
6,12 Angry Men,1957,96,9.0,96,"748,260",$4.36M,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
7,The Lord of the Rings: The Return of the King,2003,201,8.9,94,"1,747,406",$377.85M,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."
8,Pulp Fiction,1994,154,8.9,94,"1,950,392",$107.93M,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"[John Travolta, Uma Thurman, Samuel L. Jackson..."
9,Schindler's List,1993,195,8.9,94,"1,293,820",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."


In [18]:
movie_df.shape

(100, 10)